In [1]:
import os 
import sys
import glob
import numpy as np
import itk
from itk import TubeTK as ttk
from itkwidgets import view

In [2]:
# NRRD Study Name
studyname = 'C:/Users/steph/Desktop/Data/unc/HighRes-005-ctp'

# NRRD Files
directory = (studyname + '/')

# Saved NRRD Files 
directory2 = (studyname + '-Reg/')
os.mkdir(directory2)

# Mask Creation and Location
directory3 = (studyname + '-MinMax/')
os.mkdir(directory3)

pic_folder = os.listdir(directory)
pic_folder = [pic_folder for pic_folder in pic_folder if ".nii" in pic_folder]
pic_folder.sort()
print(pic_folder)
num_images = len(pic_folder)

im0Tmp = itk.imread(directory + pic_folder[int(num_images/2)], itk.F)

resample = ttk.ResampleImage.New(Input=im0Tmp,MakeIsotropic=True)
resample.Update()
im0 = resample.GetOutput()
immath = ttk.ImageMath.New(Input=im0)
immath.Blur(1)
im0Blur = immath.GetOutput()

immath.Threshold(150, 800, 1, 0)
immath.Dilate(10, 1, 0)
mask0 = immath.GetOutputUChar()
mask0Tmp = itk.GetArrayViewFromImage(mask0)
mask0Tmp[0:4,:,:] = 0
sizeZ = mask0Tmp.shape[0]
mask0Tmp[sizeZ-4:sizeZ,:,:] = 0   #No need to update mask0 since mask0Tmp is a view of mask0 (shared memory)

itk.imwrite(mask0, directory3 + 'mask.mha', compression=True)
maskObj = itk.ImageMaskSpatialObject[3].New()
maskObj.SetImage(mask0)
maskObj.Update()

['HighRes-005-08.nii', 'HighRes-005-09.nii', 'HighRes-005-10.nii', 'HighRes-005-11.nii', 'HighRes-005-12.nii', 'HighRes-005-13.nii', 'HighRes-005-14.nii', 'HighRes-005-15.nii', 'HighRes-005-16.nii', 'HighRes-005-17.nii', 'HighRes-005-18.nii', 'HighRes-005-19.nii', 'HighRes-005-20.nii', 'HighRes-005-21.nii', 'HighRes-005-22.nii', 'HighRes-005-23.nii', 'HighRes-005-24.nii', 'HighRes-005-25.nii', 'HighRes-005-26.nii', 'HighRes-005-27.nii', 'HighRes-005-28.nii', 'HighRes-005-29.nii', 'HighRes-005-30.nii', 'HighRes-005-31.nii', 'HighRes-005-32.nii', 'HighRes-005-33.nii', 'HighRes-005-34.nii', 'HighRes-005-35.nii', 'HighRes-005-36.nii', 'HighRes-005-37.nii']


In [3]:
view(mask0)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [4]:
Dimension = 3
PixelType = itk.ctype('float')
ImageType = itk.Image[PixelType, Dimension]

imdatamax = itk.GetArrayFromImage(im0)
imdatamin = imdatamax
imdatamax2 = imdatamax
imdatamin2 = imdatamax
imdatamax3 = imdatamax
imdatamin3 = imdatamax

imFixedBlur = im0Blur

for imNum in range(num_images):
    imMoving = itk.imread( directory + pic_folder[imNum], itk.F )
    
    immath.SetInput(imMoving)
    immath.Blur(1)
    imMovingBlur = immath.GetOutput()
    
    imreg = ttk.RegisterImages[ImageType].New()
    imreg.SetFixedImage(imFixedBlur)
    imreg.SetMovingImage(imMovingBlur)
    
    imreg.SetRigidMaxIterations(3000)
    imreg.SetRegistration("RIGID")
    imreg.SetExpectedOffsetMagnitude(20)
    imreg.SetExpectedRotationMagnitude(0.3)
    imreg.SetMetric("MEAN_SQUARED_ERROR_METRIC")
    
    imreg.SetFixedImageMaskObject(maskObj)
    #imreg.SetSampleFromOverlap(True)

    imreg.SetReportProgress(True)
    imreg.Update()
    
    tfm = imreg.GetCurrentMatrixTransform()
    #imFixedBlur = imreg.GetFinalMovingImage("LINEAR_INTERPOLATION", -1024)
    imMovingReg = imreg.ResampleImage("LINEAR_INTERPOLATION", imMoving, tfm, -1024)
    
    itk.imwrite( imMovingReg, directory2 + pic_folder[imNum], compression=True )
    
    print(tfm)
    
    imdataTmp = itk.GetArrayFromImage(imMovingReg)
    
    imdatamax = np.maximum(imdatamax,imdataTmp)
    imdatamin = np.minimum(imdatamin,imdataTmp)
    imdataTmp[np.where(imdataTmp==imdatamax)] = 0
    imdataTmp[np.where(imdataTmp==imdatamin)] = 0
    imdatamax2 = np.maximum(imdatamax2,imdataTmp)
    imdatamin2 = np.minimum(imdatamin2,imdataTmp)
    imdataTmp[np.where(imdataTmp==imdatamax)] = 0
    imdataTmp[np.where(imdataTmp==imdatamin)] = 0
    imdatamax3 = np.maximum(imdatamax3,imdataTmp)
    imdatamin3 = np.minimum(imdatamin3,imdataTmp)
    
    #out = itk.GetImageFromArray(imdatamax)
    #out.CopyInformation(im0)
    #itk.imwrite(out, (directory3 + 'max_' + str(imNum) + '.nrrd'))
    
    #out = itk.GetImageFromArray(imdatamax3)
    #out.CopyInformation(im0)
    #itk.imwrite(out, (directory3 + 'max3_' + str(imNum) + '.nrrd'))
    
    percent = (imNum + 1) / num_images * 100
    print(str(round(percent)) + '% : ' + pic_folder[imNum])
    
print('Done')    

AffineTransform (000001E6EABDC510)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 1473080
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.999999 -0.00108074 -0.000238296 
    0.00108011 0.999996 -0.002626 
    0.000241133 0.00262574 0.999997 
  Offset: [-0.297534, -0.26435, 0.572864]
  Center: [0.249023, -206.249, -85.0002]
  Translation: [-0.0543783, -0.0400388, 0.031664]
  Inverse: 
    0.999999 0.00108011 0.000241133 
    -0.00108074 0.999996 0.00262574 
    -0.000238296 -0.002626 0.999997 
  Singular: 0

3% : HighRes-005-08.nii
AffineTransform (000001E6ED7A0C80)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 2973625
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.999998 -0.00096391 0.00145499 
    0.000969446 0.999992 -0.00380863 
    -0.00145131 0.00381003 0.999992 
  Offset: [-0.107083, -0.322337, 0.562321]
  Center: [0.249023, -206.249, -85.0002

47% : HighRes-005-21.nii
AffineTransform (000001E6ED7A1060)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 22035419
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.999999 -0.000737246 -0.0012442 
    0.000738273 0.999999 0.000825519 
    0.0012436 -0.000826437 0.999999 
  Offset: [-0.294909, 0.0845048, 0.0648038]
  Center: [0.249023, -206.249, -85.0002]
  Translation: [-0.0370949, 0.0146459, 0.23566]
  Inverse: 
    0.999999 0.000738273 0.0012436 
    -0.000737246 0.999999 -0.000826437 
    -0.0012442 0.000825519 0.999999 
  Singular: 0

50% : HighRes-005-22.nii
AffineTransform (000001E6ED7A1630)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 23534036
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    1 -7.19628e-08 -1.40083e-06 
    7.19902e-08 1 1.96006e-05 
    1.40083e-06 -1.96006e-05 1 
  Offset: [-0.000102384, 0.00214956, -0.00839342]
  Center: [0.249

93% : HighRes-005-35.nii
AffineTransform (000001E6ED7A25B0)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 42538593
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.999979 0.00575135 -0.00286547 
    -0.0057465 0.999982 0.00169795 
    0.00287518 -0.00168145 0.999994 
  Offset: [0.422668, 0.17194, -0.199501]
  Center: [0.249023, -206.249, -85.0002]
  Translation: [-0.519982, 0.0298854, 0.148484]
  Inverse: 
    0.999979 -0.0057465 0.00287518 
    0.00575135 0.999982 -0.00168145 
    -0.00286547 0.00169795 0.999994 
  Singular: 0

97% : HighRes-005-36.nii
AffineTransform (000001E6ED7A1A10)
  RTTI typeinfo:   class itk::AffineTransform<double,3>
  Reference Count: 2
  Modified Time: 44011575
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    0.999987 0.00471643 -0.00182064 
    -0.00471358 0.999988 0.00156779 
    0.00182801 -0.00155919 0.999997 
  Offset: [0.48696, 0.175708, -0.446303]
  Center: [0.249023

In [5]:
out = itk.GetImageFromArray(imdatamax3)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'max3.nrrd'), compression=True)

out = itk.GetImageFromArray(imdatamin3)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'min3.nrrd'), compression=True)

out = itk.GetImageFromArray(imdatamax3 - imdatamin3)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'diff3.nrrd'), compression=True)

print('Done3')

Done3


In [6]:
out = itk.GetImageFromArray(imdatamax)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'max.nrrd'), compression=True)

out = itk.GetImageFromArray(imdatamin)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'min.nrrd'), compression=True)

out = itk.GetImageFromArray(imdatamax - imdatamin)
out.CopyInformation(im0)
itk.imwrite(out, (directory3 + 'diff.nrrd'), compression=True)

print('Done')

Done


In [7]:
out = itk.GetImageFromArray(imdatamax)
view(out)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…